In [1]:
import os

db_dir = os.environ["DATA"] + "PatImgXAI_data/db2.0.0/"
os.makedirs(db_dir, exist_ok=True)

test_datasets_sizes=1000
valid_datasets_sizes=1000
full_datasets_pos_samples_nb=5000
full_datasets_neg_samples_nb=5000
sample_nb_per_class = 100

In [2]:
# Number of images generated
NBGEN = 1000000

# Grid division of each image
X_DIVISIONS = 6
Y_DIVISIONS = 6

# Size of the images in pixels
img_size = (700, 700)

# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS  = ["#A33E9A", "#E0B000", "#0C90C0"] # Purple, Yellow, Blue

In [3]:
from xaipatimg.datagen.gendataset import generic_rule_exist_row_with_only_shape, generic_rule_N_times_color_exactly, \
    generic_rule_shape_color_plus_shape_equals_N, generic_rule_exist_row_with_only_color_and_col_with_only_shape

rules_data = [
    {"name": "easy_1_6_blue", "gen_fun": generic_rule_N_times_color_exactly, "gen_kwargs": {"color": "#0C90C0", "N": 6, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS}, "question": "In the image, is there exactly 6 blue symbols?"},
    {"name": "easy_2_row_circle", "gen_fun": generic_rule_exist_row_with_only_shape, "gen_kwargs": {"shape": "circle", "y_division": Y_DIVISIONS},
     "question": "In the image, is there at least one row (1, ..., 6) containing only circles?"},
    # {"name": "easy_3_7_purple", "gen_fun": generic_rule_N_times_color_exactly, "gen_kwargs": {"color": "#A33E9A", "N": 7, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS}, "question": "In the image, is there exactly 7 purple symbols?"},
    # {"name": "easy_4_row_triangle", "gen_fun": generic_rule_exist_row_with_only_shape, "gen_kwargs": {"shape": "triangle", "y_division": Y_DIVISIONS},
    #  "question": "In the image, is there at least one row (1, ..., 6) containing only triangles?"},
    # {"name": "easy_5_7_yellow", "gen_fun": generic_rule_N_times_color_exactly, "gen_kwargs": {"color": "#E0B000", "N": 5, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS}, "question": "In the image, is there exactly 7 yellow symbols?"},
    # {"name": "easy_6_row_square", "gen_fun": generic_rule_exist_row_with_only_shape, "gen_kwargs": {"shape": "square", "y_division": Y_DIVISIONS},
    #  "question": "In the image, is there at least one row (1, ..., 6) containing only squares?"},
    #
    # {"name": "hard_1_blue_square_plus_circle_8", "gen_fun": generic_rule_shape_color_plus_shape_equals_N, "gen_kwargs": {"color1": "#0C90C0", "shape1": "square", "shape2": "circle", "N": 8, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS,},
    #  "question": "In the image, does the number of blue squares plus (+) the number of circles equal to 8?"},
    # {"name": "hard_2_row_purple_col_triangle", "gen_fun": generic_rule_exist_row_with_only_color_and_col_with_only_shape, "gen_kwargs": {"color": "#A33E9A", "shape": "triangle" ,"x_division": X_DIVISIONS, "y_division": Y_DIVISIONS},
    #  "question": "In the image, is there at least one row (1, ..., 6) containing only purple symbols, and one column (A, ..., F) containing only triangles?"},
    # {"name": "hard_3_yellow_circle_plus_triangle_9", "gen_fun": generic_rule_shape_color_plus_shape_equals_N, "gen_kwargs": {"color1": "#E0B000", "shape1": "circle", "shape2": "triangle", "N": 9, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS},
    #  "question": "In the image, does the number of yellow circles plus (+) the number of triangles equal to 9?"},
    # {"name": "hard_4_row_yellow_col_circle", "gen_fun": generic_rule_exist_row_with_only_color_and_col_with_only_shape, "gen_kwargs": {"color": "#E0B000", "shape": "circle" ,"x_division": X_DIVISIONS, "y_division": Y_DIVISIONS},
    #  "question": "In the image, is there at least one row (1, ..., 6) containing only yellow symbols, and one column (A, ..., F) containing only circles?"},
    # {"name": "hard_5_purple_triangle_plus_square_7", "gen_fun": generic_rule_shape_color_plus_shape_equals_N, "gen_kwargs": {"color1": "#A33E9A", "shape1": "triangle", "shape2": "square", "N": 7, "x_division": X_DIVISIONS, "y_division": Y_DIVISIONS},
    #  "question": "In the image, does the number of purple triangles plus (+) the number of squares equal to 7?"},
    # {"name": "hard_6_row_blue_col_square", "gen_fun": generic_rule_exist_row_with_only_color_and_col_with_only_shape, "gen_kwargs": {"color": "#0C90C0", "shape": "square" ,"x_division": X_DIVISIONS, "y_division": Y_DIVISIONS},
    #  "question": "In the image, is there at least one row (1, ..., 6) containing only blue symbols, and one column (A, ..., F) containing only squares?"},
]

In [4]:
from xaipatimg.datagen.dbimg import load_db

db = load_db(db_dir)

In [5]:
import numpy as np
from xaipatimg.datagen.dbimg import generate_uuid
import os

to_generate = NBGEN
unique_content_generated = {}
duplicate_count = 0
while to_generate > 0:
    content = []
    for i in range(X_DIVISIONS):
        for j in range(Y_DIVISIONS):
            if np.random.random() < SHAPE_PROB:
                content.append({
                    "shape": np.random.choice(SHAPES),
                    "pos": (i, j),
                    "color": np.random.choice(COLORS)
                })

    if str(content) in unique_content_generated:
        duplicate_count += 1
        continue

    imgid = generate_uuid()
    db[imgid] = {
        "path": os.path.join("img", imgid + ".png"),
        "division" : (X_DIVISIONS, Y_DIVISIONS),
        "size": img_size,
        "content": content
    }

    unique_content_generated[str(content)] = True
    to_generate -= 1

print("unique generated in DB : " + str(len(db)))
print("duplicates avoided : " + str(duplicate_count))

unique generated in DB : 1000000
duplicates avoided : 0


In [6]:
import tqdm

content_dict = {}
nb_duplicates = 0

for k, v in tqdm.tqdm(db.items()):
    if str(v["content"]) in content_dict:
        nb_duplicates += 1
    else:
        content_dict[str(v["content"])] = True

print(nb_duplicates)

100%|██████████| 1000000/1000000 [00:38<00:00, 25679.41it/s]

0


In [7]:
from xaipatimg.datagen.genimg import gen_img_and_save_db
# gen_img_and_save_db(db, db_dir, overwrite=True, n_jobs=20)

100%|██████████| 1000000/1000000 [12:23<00:00, 1345.24it/s]


In [5]:
from xaipatimg.datagen.gendataset import create_dataset_generic_rule_extract_sample
import tqdm

for rule_line in tqdm.tqdm(rules_data):
    name = rule_line["name"]
    sample_path = os.path.join(db_dir, "datasets", f"{name}_train")
    create_dataset_generic_rule_extract_sample(db_dir, csv_name_train=name+"_train.csv", csv_name_test=name+"_test.csv",
                                               csv_name_valid=name+"_valid.csv", test_size=test_datasets_sizes, valid_size=valid_datasets_sizes,
                                               dataset_pos_samples_nb=full_datasets_pos_samples_nb,
                                               dataset_neg_samples_nb=full_datasets_neg_samples_nb,
                                               sample_path=sample_path, sample_nb_per_class=sample_nb_per_class,
                                               generic_rule_fun=rule_line["gen_fun"], **rule_line["gen_kwargs"])

100%|██████████| 1000000/1000000 [00:26<00:00, 38192.89it/s][A


Total number of positive instances found in database : 144633
Total number of negative instances found in database : 855367



0it [00:00, ?it/s]
25it [00:00, 247.44it/s]
61it [00:00, 310.74it/s]
93it [00:00, 271.45it/s]
121it [00:00, 266.99it/s]
157it [00:00, 297.81it/s]
8001it [00:00, 10690.08it/s]
100%|██████████| 1000000/1000000 [01:28<00:00, 11310.55it/s][A


Total number of positive instances found in database : 140854
Total number of negative instances found in database : 859146



0it [00:00, ?it/s]
25it [00:00, 239.61it/s]
49it [00:00, 203.40it/s]
75it [00:00, 222.35it/s]
100it [00:00, 231.87it/s]
130it [00:00, 250.00it/s]
156it [00:00, 248.86it/s]
8001it [00:00, 9477.96it/s]A
100%|██████████| 1000000/1000000 [00:23<00:00, 42641.16it/s][A


...


0it [00:00, ?it/s]
17it [00:00, 164.71it/s]
43it [00:00, 217.53it/s]
65it [00:00, 209.35it/s]
92it [00:00, 229.76it/s]
123it [00:00, 257.53it/s]
155it [00:00, 277.70it/s]
8001it [00:00, 9830.36it/s]A
100%|██████████| 1000000/1000000 [01:27<00:00, 11447.05it/s][A


Total number of positive instances found in database : 152648
Total number of negative instances found in database : 847352



0it [00:00, ?it/s]
20it [00:00, 196.92it/s]
55it [00:00, 285.15it/s]
84it [00:00, 261.53it/s]
111it [00:00, 251.76it/s]
137it [00:00, 241.30it/s]
162it [00:00, 235.07it/s]
8001it [00:00, 9909.75it/s]A
100%|██████████| 1000000/1000000 [00:23<00:00, 42726.97it/s][A


Total number of positive instances found in database : 143712
Total number of negative instances found in database : 856288



0it [00:00, ?it/s]
32it [00:00, 308.45it/s]
63it [00:00, 251.36it/s]
89it [00:00, 247.43it/s]
116it [00:00, 254.05it/s]
147it [00:00, 272.11it/s]
8001it [00:00, 10824.68it/s]
100%|██████████| 5/5 [07:32<00:00, 90.47s/it] 
